# 1. 타겟종이 존재하는 위치 주변의 정보 파악하기  

In [1]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
from haversine import haversine

## 1-1. 타겟종 주변의 무당벌레 종 분포 파악

In [3]:
# 무당벌레 전체 관측목록
kellog = pd.read_csv("./brookings_absence후보.csv")

In [36]:
gps =[]
for i in kellog.gps:
    gps.append((float(i.split(',')[0].replace("(","")),float(i.split(',')[1].replace(")",""))))

In [37]:
kellog.gps = gps

In [42]:
akron = []
# 아놀드 애비뉴
akron.append(df[(df.Latitude<=41.079569)&(df.Latitude>=41.079365)&(df.State=='Ohio')].index)
# 웨스트 톨매지
akron.append(df[(df.Latitude<=41.102115)&(df.Latitude>=41.101432)&(df.Longitude<=-81.515300)&(df.State=='Ohio')].index)
# 가필드 스트리트
akron.append(df[(df.Latitude<=41.111009)&(df.Latitude>=41.102003)&(df.State=='Ohio')].index)
#아이라 애비뉴
akron.append(df[(df.Longitude<=-81.529930)&(df.Longitude>=-81.543524)&(df.State=='Ohio')].index)
#문스트리트
akron.append(df[(df.Longitude<=-81.543149)&(df.Longitude>=-81.544314)&(df.State=='Ohio')].index)
# 사우스알링턴 스트리트
akron.append(df[(df.Latitude<=41.070844)&(df.Latitude>=41.054672)&(df.Longitude>=-82)&(df.State=='Ohio')].index)



In [48]:
akron_idx = []
for i in akron:
    for j in i:
        akron_idx.append(j)

In [53]:
kellog = df.iloc[akron_idx].reset_index(drop=True)

In [23]:
# 무당벌레 전체 관측목록
df = pd.read_csv("../Data/20220209_07_21_무당벌레관측.csv")



# gps 오차 1000m미만만 남기기
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)


#Latitude, Longitude 0으로 기록된 부분들 삭제
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)


#gps 변환
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df['gps'] = df_gps   

#sp.로 들어간 애들을 제거
sp_idx = []
for i in range(len(df)):
    if 'sp.' in df.Species[i]:
        sp_idx.append(i)
df= df.drop(sp_idx).reset_index(drop=True)

# value_counts 해서 통계적으로 유의미한 30개도 안되는 애들 고르기
under_30 = []
for i in range(len(df.Species.value_counts())):
    if df.Species.value_counts()[i]<30:
        under_30.append(df.Species.value_counts().index[i])
        
        
# 고른애들과 이름이 같다면 drop해야함
drop_30 = []
for i in tqdm(range(len(df.Species))):
    for j in under_30:
        if df.Species[i] == j:
            drop_30.append(i)        
df= df.drop(drop_30).reset_index(drop=True)            
            
# 속만 있는 경우 제거
one_letter = []
for i in df.Species.value_counts().index:
    if len(i.split(' ')) == 1:
        one_letter.append(i)    
        
drop_one = []
for i in tqdm(range(len(df.Species))):
    for j in one_letter:
        if df.Species[i] == j:
            drop_one.append(i)        
df = df.drop(drop_one).reset_index(drop=True)

100%|███████████████████████████████████████████████████████████████████████| 175834/175834 [00:12<00:00, 14149.35it/s]


In [54]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in df.Species.value_counts().index:

    kellog[i+'_18'] = 0

In [56]:
# 주변 종 체크
for i in tqdm(range(len(kellog))): # target만 뽑아놓은 데이터 프레임
    for j in range(len(df.gps)):  # 전체 무당벌레 데이터 프레임
        if haversine(kellog.gps[i], df.gps[j], unit='km') <= 18 and kellog.Year[i] == df.Year[j]:
            kellog[df.Species[j]+'_18'][i]+=1

  0%|                                                                                           | 0/46 [00:00<?, ?it/s]<ipython-input-56-a2d82a62ea73>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[df.Species[j]+'_18'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [02:15<00:00,  2.95s/it]


In [42]:
kellog.iloc[:,10:].describe()

,Harmonia axyridis_18,Coccinella septempunctata_18,Hippodamia convergens_18,Coelophora maculata_18,Cycloneda sanguinea_18,Propylea quatuordecimpunctata_18,Cycloneda munda_18,Psyllobora vigintimaculata_18,Chilocorus stigma_18,Olla v-nigrum_18,...,Nephus flavifrons_18,Brachiacantha quadripunctata_18,Hyperaspis proba_18,Egius platycephalus_18,Hyperaspis connectens_18,Nephus intrusus_18,Exochomus fasciatus_18,Coccinella undecimpunctata_18,Scymnus louisianae_18,Delphastus pusillus_18
count,22.000000,22.000000,22.000000,22.000000,22.0,22.0,22.000000,22.0,22.0,22.0,...,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0,22.0
mean,1.090909,0.545455,0.454545,1.318182,0.0,0.0,0.136364,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,1.150945,0.509647,0.509647,1.129111,0.0,0.0,0.351250,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.000000,1.000000,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,2.000000,1.000000,1.000000,2.500000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,3.000000,1.000000,1.000000,3.000000,0.0,0.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1-2. 타겟종 주변의 새 정보 파악

In [59]:
os.chdir(path = '../Data/E_bird/')

In [60]:
# Ebird에서 가져온 새 목록 합치기


# directory 내의 csv들을 전부 가져와서
for i in range(len(os.listdir())):
    globals()['bird{}'.format(i)] = pd.read_csv(os.listdir()[i],sep='\t')

# for_merge 라는 변수에 모두 합쳐놓음
for_merge = bird1.iloc[0:0]
for i in range(26):
    for_merge = pd.concat([for_merge,globals()['bird{}'.format(i)]])
bird2021 = pd.read_csv("../새2021.csv")
for_merge = pd.concat([for_merge, bird2021]).reset_index(drop=True)


# EBIRD자료는 GPS정확도를 제공하지 않으므로, GPS정확도가 존재한다면 INAT의 자료임
for_merge = for_merge[for_merge.coordinateUncertaintyInMeters.isnull()].reset_index(drop=True)

# 필요한 컬럼들만 남김
for_merge=for_merge[['species','year','month','decimalLatitude','decimalLongitude']].reset_index(drop=True)

# 종 이름이 NAN이면 쓸수 없으므로 DROP
for_merge=for_merge.drop(for_merge[for_merge.species.isnull()==True].index).reset_index(drop=True)

# haversine은 set에 들어있는 gps값을 받음 (lat, lon) 이형식으로 변환하는 과정
for_gps = []
for i in tqdm(range(len(for_merge))):
    for_gps.append( (for_merge['decimalLatitude'][i], for_merge['decimalLongitude'][i]) )
    
for_merge['gps'] = for_gps



for_merge=for_merge[(for_merge.species!='Locustella certhiola')&(for_merge.species!='Locustella ochotensis')&
         (for_merge.species!='Ianthocincla pectoralis')&(for_merge.species!='Luscinia calliope')&
         (for_merge.species!='Iduna aedon')&(for_merge.species!='Phylloscopus sibillatrix')&
         (for_merge.species!='Muscicapa latirostris')&(for_merge.species!='Tiaris bicolor')&
         (for_merge.species!='Tiaris canorus')&(for_merge.species!='Lonchura oryzivora')&
         (for_merge.species!='Lonchura cucullata')&(for_merge.species!='Estrilda coerulescens')&
         (for_merge.species!='alcarius ornatus')]

# 무당벌레는 관측지역에 잡힌 숫자를 믿을 수 없어서 관측 자체만 믿었음, 새 역시도 drop_duplicate로 동일한 절차 진행
for_merge = for_merge.drop_duplicates(keep='first').reset_index(drop=True)


# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (49) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\DAEGYEONGLEE\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,16,17,36,37,38,39,41,43,44,46,48) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
100%|████████████████████████████████████████████████████████████████████| 3034469/3034469 [00:27<00:00, 112272.64it/s]


In [61]:
# 타겟종의 데이터프레임에 주변 정보를 카운트해서 넣을 컬럼 생성
for i in for_merge.species.value_counts().index:
    kellog[i+'_18'] = 0

In [62]:
for i in tqdm(range(len(kellog))): # 타겟종만 뽑아놓은 데이터 프레임
    for j in range(len(for_merge.gps)):  # 전체 새 데이터 프레임
            if haversine(kellog.gps[i], for_merge.gps[j], unit='km') <= 18 and kellog.Year[i] == for_merge.year[j]:
                # 타겟종 관측 좌표와 새 관측좌표의 거리가 1km이하이고, 해당 좌표의 관측 연도가 같다면 count +1
                kellog[for_merge.species[j]+'_18'][i]+=1


  0%|                                                                                           | 0/46 [00:00<?, ?it/s]<ipython-input-62-5a383200347b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kellog[for_merge.species[j]+'_18'][i]+=1
100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [21:14<00:00, 27.71s/it]


In [71]:
kellog.drop_duplicates(keep='first').to_csv("./akron_absence_조사.csv")

In [87]:
kellog = pd.read_csv("../../일반화 과정/미주리는60부터absence_minmax.csv")

In [88]:
kellog[kellog['Coccinella novemnotata_18']==0]

,Unnamed: 0,Year,Harmonia axyridis_18,Coccinella septempunctata_18,Hippodamia convergens_18,Coelophora maculata_18,Cycloneda sanguinea_18,Propylea quatuordecimpunctata_18,Cycloneda munda_18,Psyllobora vigintimaculata_18,...,Locustella lanceolata_18,Acrocephalus schoenobaenus_18,Acrocephalus dumetorum_18,Cyanerpes cyaneus_18,Oenanthe pleschanka_18,Luscinia cyane_18,Taeniopygia guttata_18,Phoenicurus phoenicurus_18,Locustella fluviatilis_18,Pachyramphus major_18
0,0,2009,0.000000,0.000,0.00,0.000000,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,1,2009,0.000000,0.000,0.00,0.000000,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2,2,2007,0.000000,0.000,0.00,0.000000,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,3,2008,0.000000,0.000,0.00,0.000000,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,4,2010,0.000000,0.000,0.00,0.000000,0,0,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,106,2021,0.153061,0.375,0.25,0.176471,0,0,0.285714,0.000000,...,0,0,0,0,0,0,0,0,0,0
107,107,2021,0.020408,0.250,0.00,0.117647,0,0,0.142857,0.333333,...,0,0,0,0,0,0,0,0,0,0
108,108,2021,1.000000,1.000,0.25,1.000000,0,0,1.000000,1.000000,...,0,0,0,0,0,0,0,0,0,0
109,109,2021,0.071429,0.000,1.00,0.058824,0,0,0.285714,0.333333,...,0,0,0,0,0,0,0,0,0,0
